# Hartree-Fock 

## Introduction
Hartree fock method is the simplest physically relevant method of solving the time-independent Schrodinger equation for a many-body system.

The genius of this method lies in the gradual solving of the problem for individual electrons. Problematic electron-electron interaction is dealth with by considering the electron as existing in an "averaged" potential of other electrons.

## Initial approximations
Hartree Fock method makes five major simplifications/assumptions to deal with the Schrodingers equation:

- **Born-Oppenheimer approximation**: separation of nuclear and electronic movement
- **Model of independent electrons**
- **Wavefunction in a Slater determinant form**
- **MO LCAO**: Molecular orbitals as Linear Combination of Atomic Orbitals

## SCF procedure
**0. Geometry and basis set**

- **Geometry**: The positions of the nuclei determine the potential field in which the electrons move. 

- **Basis Set**: Basis set consists of a predefined set of functions $\phi$ (atomic orbitals - AO) centered on atom nuclei that are used to construct molecular orbitals (MO). These functions typically Gaussian functions, Slater-type orbitals or exact solutions from the hydrogen atom. They serve as the building block for approximating the wavefunction of the system.

**1. Starting MO - set of coefficients $c_{iA}$**

Molecular orbitals are constructed as linear combinations of atomic orbitals using MO LCAO:

$\psi_i (r) = \sum_A c_{iA} \Psi_A (r)$

where $\psi_i(r)$ is the $i$-th molecular orbital, $c_{iA}$ are coefficients weighing the contribution of each basis function (atomic orbital) $\phi_A(r)$ to the total molecular orbital.

An initial guess for the molecular orbital coefficents $c_{iA}$ provides a starting point for the iterative SCF process. This guess can be based on simpler methods like Hucker theory. These estimated coefficients are then refined throughout the SCF process thanks to the *Variational theorem* which states:

$E_0 \le \langle \psi \hat{H} \psi \rangle$

that means that the eigenvalue (energy) of any trial wavefunction is greater or equal than the true ground state energy. The implications of this are massive and allow for the iterative minimization of energy with respect to the coefficients $c_{iA}$ to yield the lowest possible energy within the chosen basis set.

**2. Calculating the integrals $\langle AB|CD \rangle$, $S_{AB}$, $h_{AB}$**
- Overlap integrals $S_{AB}$ measure the overlap between basis functions. Basis sets are generally not orthonormal. And in order to write the Fock Equations equations in the form of an matrix eigenvalue problem, we need to orthogonalize our basis, hence why including the $S$ matrix. 

    $S_{AB} = \langle \phi_A | \phi_B \rangle = \int \phi_A \phi_B dr$

- Core Hamiltonian $h_{AB}$: Represents the kinetic energy of electrons and their attraction to the nucleic. It forms the starting point for building the Fock matrix.

    $h_{AB} = \int \phi_A (r) \bigg( - \frac{1}{2} \nabla^2 - \sum_N \frac{Z_N}{|r-R_N|}\bigg) \phi_B (r) dr$

- Two electron repulsion integrals $\langle AB|CD \rangle$ account for electron electron repulsion. These integrals are crucial for incorporating electron correlation effects within the mean-field approximation

**3. Create matrix P_{AB}**

Density matrix encodes the distribution of electrons over the molecular orbitals. It is used to calculate the electron density and thus the effective potential experienced by each electron.

4) Create matrix F_{AB}

5) Solving Fock equations 

6) Checking convergence criteria - back to item 3

In [2]:
import sys
import math
import numpy as np

In [9]:
#def primitive_gaussian(alpha, coeff, coordinates, l1, l2, l3)   
#    gto = (2.0 * alpha / math.pi) ** 0.75 # + other terms for angular momentum > 0
    
class primitive_gaussian():
    
    def __init__(self, exponent, coeff, coordinates, l1, l2, l3):

        self.exponent = exponent
        self.coeff = coeff
        self.coordinates = np.array(coordinates)
        self.normalization_const = (2.0 * exponent / math.pi) ** 0.75 # + additional terms for angular momentum > 0


In [10]:
def overlap_matrix(molecule):

    n_basis = len(molecule)

    S_matrix = np.zeros([n_basis, n_basis])

    for i in range(n_basis):
        for j in range(n_basis):

            n_primitives_i = len(molecule[i])
            n_primitives_j = len(molecule[j])

            for k in range(n_primitives_i):
                for l in range(n_primitives_j):
                    
                    N = molecule[i][k].normalization_const * molecule[j][l].normalization_const
                    p = molecule[i][k].exponent + molecule[j][l].exponent
                    q = molecule[i][k].exponent * molecule[j][l].exponent / p
                    Q = molecule[i][k].coordinates - molecule[j][l].coordinates
                    Q2 = np.dot(Q,Q)

                    S_matrix[i,j] += N * molecule[i][k].coeff * molecule[j][l].coeff * math.exp(-q * Q2) * (math.pi / p)**(3/2) # + additional terms for angular momentum > 0  
    return S

In [11]:
H1_pg1a = primitive_gaussian(0.3425250914E+01, 0.1543289673E+00, [0,0,0], 0, 0, 0)
H1_pg1b = primitive_gaussian(0.6239137298E+00, 0.5353281423E+00, [0,0,0], 0, 0, 0)
H1_pg1c = primitive_gaussian(0.1688554040E+00, 0.4446345422E+00, [0,0,0], 0, 0, 0)

H2_pg1a = primitive_gaussian(0.3425250914E+01, 0.1543289673E+00, [1.2,0,0], 0, 0, 0)
H2_pg1b = primitive_gaussian(0.6239137298E+00, 0.5353281423E+00, [1.2,0,0], 0, 0, 0)
H2_pg1c = primitive_gaussian(0.1688554040E+00, 0.4446345422E+00, [1.2,0,0], 0, 0, 0)


H1_1s = [H1_pg1a, H1_pg1b, H1_pg1c]
H2_1s = [H2_pg1a, H2_pg1b, H2_pg1c]

molecule = [H1_1s, H2_1s]

ovelap_matrix(molecule)

NameError: name 'ovelap_matrix' is not defined

In [ ]:
{
    "molssi_bse_schema": {
        "schema_type": "complete",
        "schema_version": "0.1"
    },
    "revision_description": "Data from Gaussian09",
    "revision_date": "2018-06-19",
    "elements": {
        "1": {
            "electron_shells": [
                {
                    "function_type": "gto",
                    "region": "",
                    "angular_momentum": [
                        0
                    ],
                    "exponents": [
                        "0.3425250914E+01",
                        "0.6239137298E+00",
                        "0.1688554040E+00"
                    ],
                    "coefficients": [
                        [
                            "0.1543289673E+00",
                            "0.5353281423E+00",
                            "0.4446345422E+00"
                        ]
                    ]
                }
            ],
            "references": [
                {
                    "reference_description": "STO-3G Minimal Basis (3 functions/AO)",
                    "reference_keys": [
                        "hehre1969a"
                    ]
                }
            ]
        },
        "8": {
            "electron_shells": [
                {
                    "function_type": "gto",
                    "region": "",
                    "angular_momentum": [
                        0
                    ],
                    "exponents": [
                        "0.1307093214E+03",
                        "0.2380886605E+02",
                        "0.6443608313E+01"
                    ],
                    "coefficients": [
                        [
                            "0.1543289673E+00",
                            "0.5353281423E+00",
                            "0.4446345422E+00"
                        ]
                    ]
                },
                {
                    "function_type": "gto",
                    "region": "",
                    "angular_momentum": [
                        0,
                        1
                    ],
                    "exponents": [
                        "0.5033151319E+01",
                        "0.1169596125E+01",
                        "0.3803889600E+00"
                    ],
                    "coefficients": [
                        [
                            "-0.9996722919E-01",
                            "0.3995128261E+00",
                            "0.7001154689E+00"
                        ],
                        [
                            "0.1559162750E+00",
                            "0.6076837186E+00",
                            "0.3919573931E+00"
                        ]
                    ]
                }
            ],
            "references": [
                {
                    "reference_description": "STO-3G Minimal Basis (3 functions/AO)",
                    "reference_keys": [
                        "hehre1969a"
                    ]
                }
            ]
        }
    },
    "version": "1",
    "function_types": [
        "gto"
    ],
    "names": [
        "STO-3G"
    ],
    "tags": [],
    "family": "sto",
    "description": "STO-3G Minimal Basis (3 functions/AO)",
    "role": "orbital",
    "auxiliaries": {},
    "name": "STO-3G"
}